In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('Reddit_Encoded.csv')
df

,comment,hate_speech,lemmatized_comment,document_vector_flat
0,subsection retarded hungarians ohh boy brace l...,1,subsection retard hungarians ohh boy brace liv...,"0.014043219,-0.01809359,0.017145459,0.08062436..."
1,hiii just got work Foundation and grounding ma...,0,hiii just get work Foundation and ground mainl...,"-0.0030388932,-0.035133556,0.020659983,0.07383..."
2,wow guess soyboys every country,0,wow guess soyboys every country,"0.017362628,0.005587179,0.0297773,0.109146975,..."
3,owen benjamins soyboy song goes every country ...,0,owen benjamins soyboy song go every country amaze,"0.018085241,0.0011954829,2.8959475e-05,0.07601..."
4,yall hear sumn means live small town rn for w...,0,yall hear sumn mean live small town rn for wor...,"0.023993038,-0.00060867134,0.005239945,0.05989..."
...,...,...,...,...
22206,op stop faggot post videos next time hard,1,op stop faggot post videos next time hard,"0.08339707,-0.017676119,-0.036872935,0.1038081..."
22207,minute long video top hate champagne goes need...,0,minute long video top hate champagne go need g...,"0.009342635,0.02728245,-0.0013451587,0.0765217..."
22208,clue whos ecelebs are point time need get alo...,1,clue whos ecelebs be point time need get along...,"-0.0006810841,0.008315975,0.029024707,0.077029..."
22209,didn’t insult you insult me,0,didn ’ t insult you insult me,"0.06474433,-0.17082486,-0.029483724,0.04382453..."


In [3]:
df['fasttext_vector'] = df['document_vector_flat'].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))
X = np.vstack(df['fasttext_vector'].values)
y=df['hate_speech']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

### Applying SMOTE

In [6]:
from imblearn.over_sampling import SMOTE

In [7]:
smote=SMOTE(sampling_strategy='minority',random_state=42)

In [8]:
X_sm,y_sm = smote.fit_resample(X_train,y_train)
y_sm.value_counts()

C:\Users\moham\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\moham\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 199, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\moham\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\moham\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\moham\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

0    13539
1    13539
Name: hate_speech, dtype: int64

In [9]:
from sklearn.neural_network import MLPClassifier

In [11]:
mlp = MLPClassifier(max_iter=300)

In [12]:
mlp.fit(X_sm,y_sm)

C:\Users\moham\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(max_iter=300)

In [13]:
y_pred_mlp=mlp.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_score,recall_score,f1_score

In [15]:
print("mlp classifier performance:")
print("confusion matrix:\n",confusion_matrix(y_test,y_pred_mlp))
print("classification report:\n",classification_report(y_test,y_pred_mlp))

mlp classifier performance:
confusion matrix:
 [[2678  708]
 [ 228  829]]
classification report:
               precision    recall  f1-score   support

           0       0.92      0.79      0.85      3386
           1       0.54      0.78      0.64      1057

    accuracy                           0.79      4443
   macro avg       0.73      0.79      0.75      4443
weighted avg       0.83      0.79      0.80      4443



In [16]:
accuracy=accuracy_score(y_test,y_pred_mlp)
precision=precision_score(y_test,y_pred_mlp)
recall=recall_score(y_test,y_pred_mlp)
f1=f1_score(y_test,y_pred_mlp)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1:{f1:.2f}")

Accuracy:0.79
precision:0.54
recall:0.78
f1:0.64


In [17]:
mlp_gs = MLPClassifier(max_iter=100)

In [18]:
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh','sigmoid','relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}


In [20]:
from sklearn.model_selection import RandomizedSearchCV
clf = RandomizedSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)

In [21]:
clf.fit(X_sm, y_sm) # X is train samples and y is the corresponding labels

C:\Users\moham\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'alpha': [0.0001, 0.05],
                                        'hidden_layer_sizes': [(10, 30, 10),
                                                               (20,)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'solver': ['sgd', 'adam']})

In [23]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run

In [24]:
print(clf.best_params_)

{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (10, 30, 10), 'alpha': 0.0001, 'activation': 'relu'}


In [78]:
clf.best_score_

0.8838561848140525

In [10]:
mlp_rd = MLPClassifier(max_iter=100,solver='adam',learning_rate='adaptive',hidden_layer_sizes=(10, 30, 10),alpha=0.0001,activation='relu')

In [11]:
mlp_rd.fit(X_sm,y_sm)

C:\Users\moham\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(10, 30, 10), learning_rate='adaptive',
              max_iter=100)

In [12]:
y_pred_mlp=mlp_rd.predict(X_test)

In [15]:
print("mlp classifier performance:")
print("confusion matrix:\n",confusion_matrix(y_test,y_pred_mlp))
print("classification report:\n",classification_report(y_test,y_pred_mlp))

mlp classifier performance:
confusion matrix:
 [[2973  413]
 [ 313  744]]
classification report:
               precision    recall  f1-score   support

           0       0.90      0.88      0.89      3386
           1       0.64      0.70      0.67      1057

    accuracy                           0.84      4443
   macro avg       0.77      0.79      0.78      4443
weighted avg       0.84      0.84      0.84      4443



In [16]:
accuracy=accuracy_score(y_test,y_pred_mlp)
precision=precision_score(y_test,y_pred_mlp)
recall=recall_score(y_test,y_pred_mlp)
f1=f1_score(y_test,y_pred_mlp)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1:{f1:.2f}")

Accuracy:0.84
precision:0.64
recall:0.70
f1:0.67
